## Voronoi expansion implementation in DEVS Simulation Framework 


In [1]:
%load_ext autoreload
%autoreload 2

### Attemping to make an animated matplotlib celldevs plot
Using as example project a Vornoi Zones implementation in cellDevs.

See this links.
- http://louistiao.me/posts/notebooks/embedding-matplotlib-animations-in-jupyter-notebooks/
- https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.load.html
- https://www.numpy.org/devdocs/reference/generated/numpy.lib.format.html
- https://github.com/rogersce/cnpy

In [2]:
import numpy as np
import matplotlib

from IPython.display import HTML

from matplotlib import pyplot as plt

from pycdevs import cdppwrapper as cdppExceptions
from pycdevs import voronoi

from pycdevs.cdppwrapper import CDPPWrapper, DrawlogFailedException
from pycdevs.models import Model, CellValues
from pycdevs.jupyter import get_simulated_voronoi_animation

matplotlib.rcParams['figure.figsize'] = (10,8)

Snippet to generate randomly initiated cell values, for a voronoi model of a certain size

In [3]:
import random

inital_cell_values = np.zeros((200, 200), dtype=int)
get_random_i = lambda : random.randint(0, inital_cell_values.shape[0]-1)
get_random_j = lambda : random.randint(0, inital_cell_values.shape[1]-1)
current_value = 1

for _ in range(5):
    random_i = get_random_i()
    random_j = get_random_j()
    inital_cell_values[random_i, random_j] = current_value
    # print("Initiating ({0},{1}) with value {2}".format(random_i, random_j, current_value))
    current_value += 1
    
devs_initiall_cell_values = CellValues(inital_cell_values, 0)

The voronoi model works by using a rule pattern with the following idea:

- Taking into account the center position of the negihbourhood, **(0,0)**, verify if in a certain direction the following position is occupied, and if not, fill it with my **site** value. 

In [4]:
simple_voronoi_rule = "rule : {${current_cell}} 100 { (0,0)=0 and ${current_cell}!=0 and ${current_cell}!=? }"
simple_voronoi_rule_builder = voronoi.RulesetBuilder(simple_voronoi_rule)

Generating a **vornoi model** programmatically with a diamond-shaped negihbourhood

In [5]:
# diamond shaped neighbourhood
diamond_neighbours = [
    (0, -2), \
    (-1,-1), (0, -1), (1,-1), \
    (-2, 0), (-1,0), (1,0), (2, 0),\
    (-1,1), (0, 1), (1,1), \
    (0, 2)
]

diamond_neighbourhood = voronoi.Neighbourhood(diamond_neighbours)
diamond_voronoi_model = voronoi.VoronoiModel("diamondVoronoi", diamond_neighbourhood,\
                                             devs_initiall_cell_values, simple_voronoi_rule_builder).build()

Generating a **vornoi model** programmatically with a square-shaped negihbourhood

In [6]:
# diamond shaped neighbourhood
square_neighbours = [
    (-1,-1), (0, -1), (1,-1), \
    (-1,0), (1,0),\
    (-1,1), (0, 1), (1,1)
]

square_neighbourhood = voronoi.Neighbourhood(square_neighbours)
square_voronoi_model = voronoi.VoronoiModel("squareVoronoi", square_neighbourhood,\
                                             devs_initiall_cell_values, simple_voronoi_rule_builder).build()

When performing the actual simulation, the process is composed by two stages:
1. Executing the simulator for a given time.
2. Piping the log files generated into drawlog, a tool whose purpose is drawing the state of each cell in given time intervals.

The drawlog tool has been modified to dump the cell states into CSV files, in order to parse them with NumPy and generate animations.

In [7]:
def simulate_voronoi_with(model):
    wrapper = CDPPWrapper(model, '00:01:00:00')
    # for debug reasons, print CD++ executed command
    print("" + " ".join(wrapper.getArguments()))

    try:
        wrapper.run()
        drawlog_numpy_filename = wrapper.drawlog('00:00:00:100')    

    # Simulator failure
    except cdppExceptions.SimulationExecutedButFailedException:
        print(wrapper.getSimulationOutput())

    # Drawlog parsing failure
    except DrawlogFailedException as e:
        print(e.stderr.decode('ascii'))

    return drawlog_numpy_filename

In [8]:
diamomd_voronoi_results = simulate_voronoi_with(diamond_voronoi_model)
square_voronoi_results = simulate_voronoi_with(square_voronoi_model)

cd++ -m/tmp/1544308651296.ma -o/tmp/1544308651296.out -t00:01:00:00 -l/tmp/1544308651296.log
cd++ -m/tmp/1544308728409.ma -o/tmp/1544308728409.out -t00:01:00:00 -l/tmp/1544308728409.log


In summary, what it does is:

1. Load drawlog-generated numpy files, with the drawn simulation
2. Conglomerate all loaded matrixes in a single one

In [9]:
%%capture
def generate_simulation_animation(simulation_output):
    return get_simulated_voronoi_animation(simulation_output, devs_initiall_cell_values, figsize=(5,5))
diamond_animation = generate_simulation_animation(diamomd_voronoi_results)
square_animation = generate_simulation_animation(square_voronoi_results)

In [11]:
import re
# automatically count column-divs inside animations html to generate custom stylesheet
# TODO: Add a more grid-like positioning in the future
animations_html = \
     '<div class="row">' +\
     '<div class="column"><center><h3>Voronoi simulation with diamond neighbourhood</h3>' +  diamond_animation +  "</center></div>" + \
     '<div class="column"><center><h3>Voronoi simulation with square neighbourhood</h3>' +  square_animation +  "</center></div>" + \
     '</div>'
column_count = len(re.compile('class="column"').findall(animations_html))
column_styles="""
<style>
.column {
    float: left;
    width: %.2f%%;
}

/* Clear floats after the columns */
.row:after {
    content: "";
    display: table;
    clear: both;
}
</style>
""" % (100/column_count)

animations_html = column_styles + animations_html

HTML(animations_html)